# RSS Infomation Research

In [1]:
# RSS에서 xml태그 별 정보 추출
!pip install feedparser 

# 인터넷 신문기사 분석(라이브러리 내의 "Article()"사용하기 위해)
!pip install newspaper3k 

# 한국어 형태소 분석기 (주어진 문장에서 명사만 추출할 목적으로 사용)
!pip install konlpy 

# 라이브러리 불러오기

In [4]:
import feedparser # xml 태그별 정보 추출
from newspaper import Article # 인터넷 신문 기사 분석
from konlpy.tag import Okt # 한국어 형태소 분석기
from collections import Counter # 갯수 세기(TF계산용)
from operator import eq # 동일한 단어인지 여부 확인 위해 사용
from bs4 import BeautifulSoup # 글에 존재할지 모르는 html 태그 삭제를 위해 사용

# 정보 수집할 rss 링크 설정 및 정보 수집하기

##### 전자신문 (https://www.etnews.com/rss/) 의 오늘의 뉴스 부분 

##### 함수 정의

In [24]:
# 제목, 링크 수집 함수
def crawl_rss(urls):
    array_rss = [] # 모든 기사를 넣을 리스트
    titles_rss = set() # 중복된 기사를 제거하기 위한 기사 제목들 집합
    
    for url in urls: # url 순회
        print(f"[Crawl Rss] {url}") # 현재 크롤링하는 rss 링크 출력
        parser_rss = feedparser.parse(url) # 현재 url을 파싱해서, 그 결과를 저장
        for p in parser_rss.entries:
            if p.title not in titles_rss: # 중복되지 않은 기사 일 경우 if 문 실행
                array_rss.append({'title':p.title, 'link':p.link})
                titles_rss.add(p.title)
#             else:
#                 print(f"Duplicated Title : {p.title}")
    return array_rss

# 링크를 통해 제목, 본문 내용 추출
def crawl_article(url, language='ko'):
    print(f"[Crawling. . .] {url}") # 현재 크롤링하는 링크 출력
    news_paper = Article(url, language=language)
    news_paper.download()
    news_paper.parse()
    return news_paper.title, preprocessing(news_paper.text)

# 불용어 전처리
def preprocessing(text): 
    text_article = BeautifulSoup(text, 'html5lib').get_text() # html 태그 제거하기
    return text_article

In [25]:
# 정보 수집할 rss 링크 모음
urls = ['http://rss.etnews.com/Section901.xml',
        'http://rss.etnews.com/Section902.xml',
        'http://rss.etnews.com/Section903.xml',
        'http://rss.etnews.com/Section904.xml']

# 제목과 기사 링크 수집하기
list_articles = crawl_rss(urls)

# 수집한 (제목,기사링크)를 토대로 제목과 본문 내용 수집하기
for article in list_articles:
    _, text = crawl_article(article['link'])
    article['text'] = text

[Crawl Rss] http://rss.etnews.com/Section901.xml
[Crawl Rss] http://rss.etnews.com/Section902.xml
[Crawl Rss] http://rss.etnews.com/Section903.xml
[Crawl Rss] http://rss.etnews.com/Section904.xml
[Crawling. . .] https://www.etnews.com/20230926000130
[Crawling. . .] https://www.etnews.com/20230926000183
[Crawling. . .] https://www.etnews.com/20230926000128
[Crawling. . .] https://www.etnews.com/20230926000101
[Crawling. . .] https://www.etnews.com/20230926000070
[Crawling. . .] https://www.etnews.com/20230926000033
[Crawling. . .] https://www.etnews.com/20230926000022
[Crawling. . .] https://www.etnews.com/20230924000164
[Crawling. . .] https://www.etnews.com/20230926000288
[Crawling. . .] https://www.etnews.com/20230926000275
[Crawling. . .] https://www.etnews.com/20230926000231
[Crawling. . .] https://www.etnews.com/20230926000185
[Crawling. . .] https://www.etnews.com/20230926000069
[Crawling. . .] https://www.etnews.com/20230926000282
[Crawling. . .] https://www.etnews.com/202309260

# 수집 내용 확인하기

In [32]:
list_articles[0]

{'title': '[사설]하반기 실적반등 기회 살리자',
 'link': 'https://www.etnews.com/20230926000130',
 'text': '국내 대표 기업 삼성전자와 LG전자가 경기침체 속에서도 3분기 실적에서 선방할 것이라는 전망이 나온다. 지난 상반기 최악의 시즌을 보낸 삼성전자는 더디지만 회복세가 예상된다. 반도체 부문 부진이 지속되지만 전 분기에 비해서는 나아질 것으로 관측된다. 상대적으로 상반기에 잘 대응한 LG전자는 하반기에도 긍정적인 흐름을 이어갈 것으로 보인다.\n\n\n\n국내외 경제 환경이 녹록지 않은 가운데 우리나라를 대표하는 두 기업이 안정을 되찾는 것은 희소식이다. 두 기업은 4분기에도 수익 개선과 회복을 지속할 것으로 기대된다. 경기 침체에 대비해 생산량 조정, 신사업 확대 등에 힘쓴 결과다.\n\n\n\n물론 안심할 상황은 아니다. 미국·중국 갈등에 따른 변수가 여전하고 국내 소비심리 회복도 뚜렷하지 않다. 한국경제인협회(한경협)가 26일 내놓은 기업경기실사지수(BSI) 조사 결과에 따르면 10월 전망치는 전월 대비 6.3포인트(P) 하락했다. BSI 전망치는 19개월째 기준치를 하회했다. 한경협은 생산·소비·투자 트리플 약세가 나타나고 있다면서 경기반등의 불확실성을 우려했다.\n\n\n\n하반기 우리 기업의 고른 실적 반등을 위해서는 보다 공격적인 대응이 필요하다. 새로운 성장 기회를 한발 앞서 발굴하는데 역량을 집중해야 한다. 정부는 기업이 하반기 반등 기회를 살릴 수 있도록 경제 활력을 되살리는데 지원과 투자를 아끼지 말아야 한다. 끊임없는 규제 개선 작업으로 기업이 반등의 날개를 달도록 도와야 한다.\n\n\n\n연내 경기 회복세에 대한 기대감이 높은 한편 침체가 지속될 것이라는 우려도 여전한 상황이다. 기업과 정부가 활발한 소통과 협업으로 경제를 활성화하는데 힘을 모아야 한다.',
 'keywords': [{'keyword': '기업', 'count': 8},
  {'keyword': '것', 'c

# 모든 본문 text에서 명사  (키워드, 빈도수) 추출

##### 키워드와 빈도수 추출하는 함수 정의

In [27]:
def get_keywords_and_frequency(text,nKeywords=10): #텍스트와  추출할 키워드 개수
    spliter = Okt() # Okt 객체 생성
    nouns = spliter.nouns(text) # 명사 추출 
    count = Counter(nouns) # 명사별 빈도수 추출
    list_keywords = []
    
    for n,c in count.most_common(nKeywords): # 최빈값 높은 명사 순으로, nKeywords까지 출력
        item = {'keyword':n, 'count':c}
        list_keywords.append(item)
    return list_keywords

##### 추출 내용 저장하기

In [28]:
for article in list_articles:
    keywords = get_keywords_and_frequency(article['text'])
    article['keywords'] = keywords

# 검색어를 입력받아서 그 검색어를 가지고 있는 기사를 출력하기

In [29]:
# 검색어와 동일한 키워드가 있을 때, 빈도수 반환
def search_articles(query, list_keywords): # 쿼리가 키워드 리스트에 있으면 출력 (없으면 0)
    nWords = 0 # 키워드 빈도수 초기화
    for kw in list_keywords:
        if eq(query, kw['keyword']):
            nWords = kw['count']
    return nWords

In [30]:
query = input("키워드를 입력하세요 : ")

for article in list_articles:
    nQuery = search_articles(query, article['keywords'])
    if nQuery != 0:
        print(f"[TF] {nQuery}, [Title] {article['title']}, [URL] {article['link']}")

키워드를 입력하세요 : 반도체
[TF] 20, [Title] 전자신문 테크서밋 내달 18일 개막…반도체·디스플레이 미래기술 엿본다, [URL] https://www.etnews.com/20230926000282
[TF] 10, [Title] [에듀플러스]제2회 도쿄일렉트론코리아 드림업 반도체 캠프, 10월 21일·11월 25일 개최…무료 참가 신청은 이티에듀 홈페이지, [URL] https://www.etnews.com/20230925000316
[TF] 25, [Title] "中서 현상 유지만 하라는 것…첨단 반도체 대규모 투자 어려워", [URL] https://www.etnews.com/20230924000040
[TF] 8, [Title] 美, '반도체법' 가드레일 규정 최종 발표, [URL] https://www.etnews.com/20230923000006
[TF] 6, [Title] 펨트론, 웨이퍼 표면 검사 장비 국산화, [URL] https://www.etnews.com/20230925000191
[TF] 10, [Title] “美 반도체 보조금 받으면 중국 내 생산능력 5%만 확장 가능”, [URL] https://www.etnews.com/20230922000447


In [31]:
list_articles[0]

{'title': '[사설]하반기 실적반등 기회 살리자',
 'link': 'https://www.etnews.com/20230926000130',
 'text': '국내 대표 기업 삼성전자와 LG전자가 경기침체 속에서도 3분기 실적에서 선방할 것이라는 전망이 나온다. 지난 상반기 최악의 시즌을 보낸 삼성전자는 더디지만 회복세가 예상된다. 반도체 부문 부진이 지속되지만 전 분기에 비해서는 나아질 것으로 관측된다. 상대적으로 상반기에 잘 대응한 LG전자는 하반기에도 긍정적인 흐름을 이어갈 것으로 보인다.\n\n\n\n국내외 경제 환경이 녹록지 않은 가운데 우리나라를 대표하는 두 기업이 안정을 되찾는 것은 희소식이다. 두 기업은 4분기에도 수익 개선과 회복을 지속할 것으로 기대된다. 경기 침체에 대비해 생산량 조정, 신사업 확대 등에 힘쓴 결과다.\n\n\n\n물론 안심할 상황은 아니다. 미국·중국 갈등에 따른 변수가 여전하고 국내 소비심리 회복도 뚜렷하지 않다. 한국경제인협회(한경협)가 26일 내놓은 기업경기실사지수(BSI) 조사 결과에 따르면 10월 전망치는 전월 대비 6.3포인트(P) 하락했다. BSI 전망치는 19개월째 기준치를 하회했다. 한경협은 생산·소비·투자 트리플 약세가 나타나고 있다면서 경기반등의 불확실성을 우려했다.\n\n\n\n하반기 우리 기업의 고른 실적 반등을 위해서는 보다 공격적인 대응이 필요하다. 새로운 성장 기회를 한발 앞서 발굴하는데 역량을 집중해야 한다. 정부는 기업이 하반기 반등 기회를 살릴 수 있도록 경제 활력을 되살리는데 지원과 투자를 아끼지 말아야 한다. 끊임없는 규제 개선 작업으로 기업이 반등의 날개를 달도록 도와야 한다.\n\n\n\n연내 경기 회복세에 대한 기대감이 높은 한편 침체가 지속될 것이라는 우려도 여전한 상황이다. 기업과 정부가 활발한 소통과 협업으로 경제를 활성화하는데 힘을 모아야 한다.',
 'keywords': [{'keyword': '기업', 'count': 8},
  {'keyword': '것', 'c